# Финальный ноутбук проекта

### Описание данных

GA Sessions:
###### Одна строка = один визит на сайт.
- session_id — ID визита;
- client_id — ID посетителя;
- visit_date — дата визита;
- visit_time — время визита;
- visit_number — порядковый номер визита клиента;
- utm_source — канал привлечения;
- utm_medium — тип привлечения;
- utm_campaign — рекламная кампания;
- utm_keyword — ключевое слово;
- device_category — тип устройства;
- device_os — ОС устройства;
- device_brand — марка устройства;
- device_model — модель устройства;
- device_screen_resolution — разрешение экрана;
- device_browser — браузер;
- geo_country — страна;
- geo_city — город

GA Hits:
###### Одна строка = одно событие в рамках одного визита на сайтv
- session_id — ID визита;
- hit_date — дата события;
- hit_time — время события;
- hit_number — порядковый номер события в рамках сессии;
- hit_type — тип события;
- hit_referer — источник события;
- hit_page_path — страница события;
- event_category — тип действия;
- event_action — действие;
- event_label — тег действия;
- event_value — значение результата действия

### Импорт сторонних библиотек

In [89]:
import re
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample

### Загрузка данных

In [2]:
com_df = pd.read_csv('data/new_df.csv')

In [3]:
com_df.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target_action
0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust,0
1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow,0
2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk,0
3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,0
4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,0


In [4]:
com_df.shape

(1781974, 19)

### Data Preparation

##### 1. Список колонок датасета

In [5]:
com_df.columns

Index(['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'utm_keyword', 'device_category', 'device_os', 'device_brand',
       'device_model', 'device_screen_resolution', 'device_browser',
       'geo_country', 'geo_city', 'target_action'],
      dtype='object')

##### 2. Описательные статистики датасета

In [6]:
com_df.describe(include=[object])

,session_id,visit_date,visit_time,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
count,1781974,1781974,1781974,1781893,1781974,1579596,1469230,727076,1781974,734010,1423571,15402,1781974,1781974,1781974,1781974
unique,1732266,226,85032,280,55,406,280,1192,3,13,200,104,4947,55,159,2389
top,1906362532065501532.1625350494.1625350494,2021-05-24,13:00:00,ZpYIoDJMcFzVoPFsHGJL,banner,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Apple,AuMdmADEIoPXiWpTsBEj,414x896,Chrome,Russia,Moscow
freq,2,41746,60306,568527,540289,441701,968986,477062,1406765,434929,517859,9226,159495,978405,1730983,774214


In [7]:
com_df.describe(exclude=[object])

,client_id,visit_number,target_action
count,1.781974e+06,1.781974e+06,1.781974e+06
mean,1.074618e+09,2.676033e+00,2.823498e-02
std,6.201745e+08,1.223985e+01,1.656435e-01
min,2.321640e+02,1.000000e+00,0.000000e+00
25%,5.378789e+08,1.000000e+00,0.000000e+00
50%,1.074768e+09,1.000000e+00,0.000000e+00
75%,1.612408e+09,2.000000e+00,0.000000e+00
max,2.147483e+09,5.640000e+02,1.000000e+00


In [8]:
com_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781974 entries, 0 to 1781973
Data columns (total 19 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   session_id                object 
 1   client_id                 float64
 2   visit_date                object 
 3   visit_time                object 
 4   visit_number              int64  
 5   utm_source                object 
 6   utm_medium                object 
 7   utm_campaign              object 
 8   utm_adcontent             object 
 9   utm_keyword               object 
 10  device_category           object 
 11  device_os                 object 
 12  device_brand              object 
 13  device_model              object 
 14  device_screen_resolution  object 
 15  device_browser            object 
 16  geo_country               object 
 17  geo_city                  object 
 18  target_action             int64  
dtypes: float64(1), int64(2), object(16)
memory usage: 258.3+ MB


Проверка на дубликаты

In [9]:
com_df[com_df.duplicated()]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target_action




Список всех признаков с пропущенными значениями:

In [5]:
missing_values = ((com_df.isna().sum() / len(com_df)) * 100).sort_values(ascending=False)
print([column for column in missing_values[missing_values != 0].keys()])

['device_model', 'utm_keyword', 'device_os', 'device_brand', 'utm_adcontent', 'utm_campaign', 'utm_source']


Пропуски в device_model

In [6]:
print(f"Количество пропущенных значений: {com_df.device_model.isna().sum()}")
com_df[com_df['device_model'].isna()].head()

Количество пропущенных значений: 1766572


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target_action
0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust,0
1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow,0
2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk,0
3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,0
4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,0


In [7]:
com_df['device_model'].value_counts(dropna=False)

device_model
NaN                     1766572
AuMdmADEIoPXiWpTsBEj       9226
tWBQlsvNfHxRUjaPAfhd        614
cwMJxNXiWUgMUxGiCTPs        596
pTgAEPipQxDXCjPrJbHo        435
                         ...   
OJiWyBKOyDITzXCZRSMH          1
MBGYWAQSYWUphNxTsAWD          1
XnjPzKjkHmznVfULanbE          1
VDidzTqFGxuqiRQJGrwB          1
qmRODeCJLlmkmwxNYXvp          1
Name: count, Length: 105, dtype: int64

In [8]:
com_df[
    (com_df['device_model'].notna()) & (com_df['device_brand'].notna())
    ]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target_action
60,9055656589238457290.1635504877.1635504877,2.108434e+09,2021-10-29,13:54:37,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Meizu,qBRdfuuhOnnqwSqNiPOv,360x744,Chrome,Russia,Moscow,0
63,905565998839028208.1635310063.1635310063,2.108435e+08,2021-10-27,07:47:43,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,OnePlus,AshpvdJdReafUzEMmWGr,384x832,Android Webview,Russia,Saint Petersburg,0
64,905565998839028208.1636951969.1636951969,2.108435e+08,2021-11-15,07:52:49,2,ISrKoXQCxqqYvAZICvjs,smm,NaN,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,OnePlus,AshpvdJdReafUzEMmWGr,384x832,Android Webview,Russia,Saint Petersburg,0
164,9056202067269505745.1640092368.1640092368,2.108561e+09,2021-12-21,16:12:48,1,ZpYIoDJMcFzVoPFsHGJL,push,sbJRYgVfvcnqKJNDDYIr,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,384x854,Chrome,Russia,Moscow,0
206,9056422519349747445.1638964982.1638964982,2.108613e+09,2021-12-08,15:03:02,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,RrhnkuoaqckNtJpAZDzH,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,320x640,Chrome,Russia,Saint Petersburg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781294,9052658994125991465.1638271992.1638271992,2.107736e+09,2021-11-30,14:33:12,3,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,DBHgBJHOdbPwsRDUbEgX,mobile,Android,Vivo,cwMJxNXiWUgMUxGiCTPs,360x760,Chrome,Russia,Moscow,0
1781299,9052676375860711226.1639943995.1639943995,2.107740e+09,2021-12-19,22:59:55,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Izhevsk,0
1781342,9052904464392788371.1638799763.1638799763,2.107794e+09,2021-12-06,17:09:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Nokia,aCLaTVQlHcXbUnQYQvIg,412x892,Chrome,Russia,Novosibirsk,0
1781547,9053730012943175107.1640761072.1640761072,2.107986e+09,2021-12-29,09:57:52,2,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,360x760,Chrome,Russia,Vladivostok,0


В device_model слишком много пропущенных значений, решил просто удалить

In [9]:
com_df.drop(columns=['device_model'], axis=1, inplace=True)
com_df

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,360x720,Chrome,Russia,Zlatoust,0
1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,385x854,Samsung Internet,Russia,Moscow,0
2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,360x720,Chrome,Russia,Krasnoyarsk,0
3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,393x786,Chrome,Russia,Moscow,0
4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,393x786,Chrome,Russia,Moscow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781969,9055415581448263752.1640159305.1640159305,2.108378e+09,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,NaN,1920x1080,Chrome,Russia,Moscow,0
1781970,9055421130527858185.1622007305.1622007305,2.108380e+09,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,390x844,Safari,Russia,Stavropol,0
1781971,9055422955903931195.1636979515.1636979515,2.108380e+09,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,375x667,Safari,Russia,Moscow,0
1781972,905543020766873816.1638189404.1638189404,2.108382e+08,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Chelyabinsk,0


In [10]:
com_df['target_action'].value_counts()

target_action
0    1731660
1      50314
Name: count, dtype: int64

In [11]:
com_df['device_os'].value_counts(dropna=False)

device_os
NaN              1047964
Android           434929
iOS               186484
Windows            83321
Macintosh          24156
Linux               4706
(not set)            309
Chrome OS             65
BlackBerry            24
Tizen                  7
Firefox OS             3
Nokia                  2
Samsung                2
Windows Phone          2
Name: count, dtype: int64

In [16]:
com_df['device_brand'].value_counts(dropna=False)

device_brand
Apple        517859
NaN          358403
Samsung      321626
Xiaomi       275796
Huawei       178330
              ...  
Star              1
Mobiistar         1
Smartfren         1
Tanix             1
Maxvi             1
Name: count, Length: 201, dtype: int64

In [17]:
com_df[com_df['device_brand'].isna()]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
28,9055505230298952295.1638478433.1638478433,2.108399e+09,2021-12-02,23:53:53,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,NaN,1536x864,Chrome,Russia,Balashikha,0
61,9055657327967035032.1629707931.1629707931,2.108435e+09,2021-08-23,11:00:00,1,nSReTmyFtbSjlPrTKoaX,banner,BVKxkCOHKUOvkpbrLMgZ,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,YaBrowser,Russia,Moscow,0
62,905565977351442956.1622561294.1622561294,2.108435e+08,2021-06-01,18:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,Chrome,Russia,Moscow,0
65,9055678214400253418.1636965866.1636965866,2.108439e+09,2021-11-15,11:44:26,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,NaN,1920x1080,Chrome,Russia,Saint Petersburg,0
72,9055788191321875859.1626006934.1626006934,2.108465e+09,2021-07-11,15:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1280x720,Chrome,Russia,Moscow,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781958,9055363711117247375.1629176721.1629176721,2.108366e+09,2021-08-17,08:00:00,1,PlbkrSYoHuZBWfYjYnfw,cpm,FTjNLDyTrXaWYgZymFkV,TuyPWsGQruPMpKvRxeBF,NaN,desktop,NaN,NaN,1920x1080,Chrome,Russia,(not set),0
1781960,9055376699099939975.1630766214.1630766214,2.108369e+09,2021-09-04,17:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,Chrome,Russia,Khimki,0
1781961,9055376699099939975.1630766214.1630766214,2.108369e+09,2021-09-04,17:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,Chrome,Russia,Khimki,1
1781969,9055415581448263752.1640159305.1640159305,2.108378e+09,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,NaN,1920x1080,Chrome,Russia,Moscow,0


In [12]:
devices_xiaomi_notnan = com_df[(com_df['device_brand'] == 'Xiaomi') & (com_df['device_os'].notna())]
devices_xiaomi_notnan

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
31,9055507467976770564.1638335492.1638335492,2.108400e+09,2021-12-01,08:11:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Saint Petersburg,0
41,9055541209241593296.1640080849.1640080849,2.108407e+09,2021-12-21,13:00:49,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Android Webview,Russia,Novorossiysk,0
58,905565212864092591.1639650867.1639650867,2.108433e+08,2021-12-16,13:34:27,2,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Tula,0
59,905565212864092591.1639652819.1639652819,2.108433e+08,2021-12-16,14:06:59,3,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Tula,0
68,9055768546153401681.1637944657.1637944657,2.108460e+09,2021-11-26,19:37:37,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Yekaterinburg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781938,9055282415982183461.1635091492.1635091492,2.108347e+09,2021-10-24,19:04:52,1,kjsLglQLzykiRbcDiGcD,cpc,bJJuEXRheRIxXEaYIXqM,NaN,KCcEkEaKEtUilBVMoCAi,mobile,Android,Xiaomi,375x833,Chrome,Russia,Saint Petersburg,0
1781943,9055315461464429378.1638961956.1638961956,2.108355e+09,2021-12-08,14:12:36,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,360x800,Chrome,Russia,Vladivostok,0
1781954,9055349000865826584.1640744740.1640744740,2.108363e+09,2021-12-29,05:25:40,1,TxKUcPpthBDPieTGmVhx,cpc,FTjNLDyTrXaWYgZymFkV,LcGIUNPUAmXtQJaDfFBR,NnplfljjtYPiMnRvogpA,mobile,Android,Xiaomi,464x1123,Chrome,Russia,Moscow,0
1781957,9055355469082180480.1636350848.1636350848,2.108364e+09,2021-11-08,08:54:08,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x873,Chrome,Russia,Moscow,0


In [13]:
com_df.loc[(com_df['device_brand'] == 'Xiaomi') & (com_df['device_os'].isna()), 'device_os'] = 'Android'

In [14]:
xiaomi_na = com_df[(com_df['device_brand'] == 'Xiaomi') & (com_df['device_os'].isna())]
xiaomi_na

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action


In [15]:
devices_huawei_notnan = com_df[(com_df['device_brand'] == 'Huawei') & (com_df['device_os'].isna())]
devices_huawei_notnan.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
46,9055578871792078746.1622347676.1622347676,2.108416e+09,2021-05-30,07:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,XKsYZiUFcdkUXQpoLKyS,NaN,mobile,NaN,Huawei,360x780,Opera,Russia,Saint Petersburg,0
69,9055768928391199041.1623652678.1623652678,2.108461e+09,2021-06-14,09:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Huawei,360x780,Chrome,Russia,Moscow,0
70,9055768928391199041.1623652678.1623652678,2.108461e+09,2021-06-14,09:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Huawei,360x780,Chrome,Russia,Moscow,1
120,9055964456785228161.1631531397.1631531596,2.108506e+09,2021-09-13,14:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Huawei,360x780,Chrome,Russia,Saint Petersburg,0
128,90560185929716668.1629639613.1629639679,2.108519e+07,2021-08-22,16:00:00,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Huawei,360x800,Opera,Russia,Moscow,0


In [16]:
com_df.loc[(com_df['device_brand'] == 'Huawei') & (com_df['device_os'].isna()), 'device_os'] = 'Android'

In [17]:
samsung_na = com_df[(com_df['device_brand'] == 'Samsung') & (com_df['device_os'].notna())]
samsung_na.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,385x854,Samsung Internet,Russia,Moscow,0
13,9055469620715506713.1635878177.1635878177,2.108391e+09,2021-11-02,21:36:17,3,gVRrcxiDQubJiljoTbGm,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Android Webview,Russia,Sochi,0
14,9055469620715506713.1636559415.1636559415,2.108391e+09,2021-11-10,18:50:15,4,SzZERoLMmrEUEhDaYcyN,cpc,NaN,NaN,NaN,mobile,Android,Samsung,412x869,Android Webview,Russia,Saint Petersburg,0
27,9055504865227535872.1639282176.1639282176,2.108399e+09,2021-12-12,07:09:36,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,okTXSMadDkjvntEHzIjp,mobile,Android,Samsung,385x854,Chrome,Russia,Moscow,0
36,9055525588441989124.1636532229.1636532229,2.108404e+09,2021-11-10,11:17:09,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x892,Chrome,Russia,Astrakhan,0


In [18]:
com_df.loc[(com_df['device_brand'] == 'Samsung') & (com_df['device_os'].isna()), 'device_os'] = 'Android'

In [19]:
com_df[(com_df['device_brand'].notna()) & (com_df['device_os'].isna())]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
5,9055447192389856083.1622453074.1622453074,2.108386e+09,2021-05-31,12:00:00,1,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,375x812,Safari,Russia,Saint Petersburg,0
32,9055511191703531814.1628451110.1628451110,2.108400e+09,2021-08-08,22:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,tablet,NaN,Apple,375x667,Safari (in-app),Russia,Moscow,0
33,9055511191703531814.1628451239.1628451239,2.108400e+09,2021-08-08,22:00:00,2,klTrhUaShgnjIbaPmqjc,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,tablet,NaN,Apple,375x667,Safari (in-app),Russia,Moscow,0
34,90555135045918912.1627381953.1627381953,2.108401e+07,2021-07-27,13:00:00,1,ghoaGAksqhKomdFrxgyJ,cpm,FTjNLDyTrXaWYgZymFkV,NhvfEqcSTGEZKxxvUZlj,NaN,mobile,NaN,Apple,375x812,Safari (in-app),Russia,Vladivostok,0
44,9055561545897976496.1626317488.1626317488,2.108412e+09,2021-07-15,05:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,MXqmDyetMTICSSitTjWV,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,428x926,Safari,United States,Sunny Isles Beach,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781947,905532652706692816.1629060816.1629060816,2.108358e+08,2021-08-15,23:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,428x926,Safari (in-app),Russia,Moscow,0
1781948,905532652706692816.1629091708.1629091708,2.108358e+08,2021-08-16,08:00:00,2,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,428x926,Safari (in-app),Russia,Moscow,0
1781955,9055349030922605117.1632752193.1632752193,2.108363e+09,2021-09-27,17:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,NaN,BQ,640x360,YaBrowser,Russia,Saint Petersburg,0
1781967,9055397194683347295.1630237022.1630237022,2.108374e+09,2021-08-29,14:00:00,1,ISrKoXQCxqqYvAZICvjs,blogger_stories,zfwIehuEfWYdYrEZgRLo,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,414x896,Safari,Russia,Zheleznodorozhny,0


In [20]:
com_df.loc[(com_df['device_brand'].notna()) & (com_df['device_os'].isna()), 'device_os'] = 'other_os'

In [22]:
com_df[(com_df['device_brand'].isna()) & (com_df['device_os'] == 'Windows')].head(5)

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
28,9055505230298952295.1638478433.1638478433,2.108399e+09,2021-12-02,23:53:53,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,NaN,1536x864,Chrome,Russia,Balashikha,0
65,9055678214400253418.1636965866.1636965866,2.108439e+09,2021-11-15,11:44:26,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,NaN,1920x1080,Chrome,Russia,Saint Petersburg,0
80,9055795561498027164.1638278300.1638278300,2.108467e+09,2021-11-30,16:18:20,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,jpZoxCaowxXvglZVUJyq,desktop,Windows,NaN,1680x1050,Chrome,Russia,Moscow,0
89,9055853019573035768.1640801016.1640801016,2.108480e+09,2021-12-29,21:03:36,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,NaN,1024x820,YaBrowser,Russia,Moscow,0
99,9055873197324118026.1635526668.1635526668,2.108485e+09,2021-10-29,19:57:48,1,jaSOmLICuBzCFqHfBdRg,email,bxOTvPtyGSdUrbwoXCPO,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,NaN,1920x1080,YaBrowser,Russia,Moscow,0


In [23]:
devices_apple_notnan = com_df[
    (com_df['device_brand'] == 'Apple') & (com_df['device_category'] == 'mobile') & (com_df['device_os'].notna())
    ]
devices_apple_notnan.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
5,9055447192389856083.1622453074.1622453074,2.108386e+09,2021-05-31,12:00:00,1,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,other_os,Apple,375x812,Safari,Russia,Saint Petersburg,0
8,9055462349345527315.1638536723.1638536723,2.108389e+09,2021-12-03,16:05:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,390x844,Safari,Russia,Moscow,0
16,9055487268745225369.1637983385.1637983385,2.108395e+09,2021-11-27,06:23:05,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,320x568,Safari,Russia,Saint Petersburg,0
18,9055488359664698205.1635763037.1635763037,2.108395e+09,2021-11-01,13:37:17,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,320x693,Safari,Russia,(not set),0
34,90555135045918912.1627381953.1627381953,2.108401e+07,2021-07-27,13:00:00,1,ghoaGAksqhKomdFrxgyJ,cpm,FTjNLDyTrXaWYgZymFkV,NhvfEqcSTGEZKxxvUZlj,NaN,mobile,other_os,Apple,375x812,Safari (in-app),Russia,Vladivostok,0


In [24]:
com_df[(com_df['device_brand'].isna()) & (com_df['device_os'].isna()) & (com_df['device_category'] == 'desktop')]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
61,9055657327967035032.1629707931.1629707931,2.108435e+09,2021-08-23,11:00:00,1,nSReTmyFtbSjlPrTKoaX,banner,BVKxkCOHKUOvkpbrLMgZ,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,YaBrowser,Russia,Moscow,0
62,905565977351442956.1622561294.1622561294,2.108435e+08,2021-06-01,18:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,Chrome,Russia,Moscow,0
72,9055788191321875859.1626006934.1626006934,2.108465e+09,2021-07-11,15:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1280x720,Chrome,Russia,Moscow,0
73,9055788191321875859.1627377569.1627377569,2.108465e+09,2021-07-27,12:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,nSReTmyFtbSjlPrTKoaX,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,Chrome,Russia,Moscow,0
78,9055794766915104430.1624305327.1624305327,2.108467e+09,2021-06-21,22:00:00,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,NaN,desktop,NaN,NaN,834x1112,Safari,Russia,Domodedovo,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781952,9055343778173115129.1628265209.1628265209,2.108362e+09,2021-08-06,18:00:00,1,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,Firefox,Russia,Moscow,0
1781953,9055345397369530035.1622009529.1622009529,2.108362e+09,2021-05-26,09:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1536x864,Firefox,Russia,(not set),0
1781958,9055363711117247375.1629176721.1629176721,2.108366e+09,2021-08-17,08:00:00,1,PlbkrSYoHuZBWfYjYnfw,cpm,FTjNLDyTrXaWYgZymFkV,TuyPWsGQruPMpKvRxeBF,NaN,desktop,NaN,NaN,1920x1080,Chrome,Russia,(not set),0
1781960,9055376699099939975.1630766214.1630766214,2.108369e+09,2021-09-04,17:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,NaN,NaN,1920x1080,Chrome,Russia,Khimki,0


In [25]:
com_df.loc[(com_df['device_brand'].isna()) & (com_df['device_os'].isna()) & (com_df['device_category'] == 'desktop'), 'device_os'] = 'Windows'

In [26]:
com_df[(com_df['device_brand'].isna()) & (com_df['device_os'].isna())]

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
23009,9161436681631382037.1625242134.1625242134,2.133063e+09,2021-07-02,19:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,JwYIveaHVpeeRZloQCfF,NaN,NaN,mobile,NaN,NaN,1440x900,Chrome,Russia,Moscow,0
25623,9172914908724584214.1634328343.1634328343,2.135736e+09,2021-10-15,23:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,hJvOVTMdhkqIBqjVrsOL,NaN,NaN,mobile,NaN,NaN,393x851,Chrome,Russia,Saint Petersburg,0
34860,9216488485801809259.1622267243.1622267243,2.145881e+09,2021-05-29,08:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,XKsYZiUFcdkUXQpoLKyS,NaN,tablet,NaN,NaN,820x1180,Safari,Russia,Krasnogorsk,0
41245,944703987097397663.1624631711.1624631711,2.199560e+08,2021-06-25,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,DnEUulZAecfGPvdtZBYS,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,NaN,393x851,Chrome,Russia,Moscow,0
57674,1021042336384541670.1624630248.1624630248,2.377299e+08,2021-06-25,17:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,DnEUulZAecfGPvdtZBYS,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,NaN,393x851,Chrome,Russia,Fryazino,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773912,9017570077669677893.1622236271.1622236271,2.099567e+09,2021-05-29,00:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,iNaIAFVuZgYukekFQQbX,vIKILzdGQEEbEpgPefxF,NaN,tablet,NaN,NaN,960x600,Samsung Internet,Russia,Saint Petersburg,0
1775182,9023760182105153779.1622221044.1622221044,2.101008e+09,2021-05-28,19:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,zPJpddwzkFqLMSYgtDqy,aYAcKhelKzYpXrRYknSP,NaN,tablet,NaN,NaN,768x1024,Safari,Russia,Saint Petersburg,0
1775425,9024892064901457390.1622238706.1622238706,2.101271e+09,2021-05-29,00:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,XKsYZiUFcdkUXQpoLKyS,NaN,mobile,NaN,NaN,360x640,Chrome,Russia,Saint Petersburg,0
1776562,9030278666862206423.1629925388.1629925388,2.102526e+09,2021-08-26,00:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,NaN,640x360,Chrome,Russia,Saint Petersburg,0


In [27]:
com_df.loc[(com_df['device_brand'].isna()) & (com_df['device_os'].isna()), 'device_os'] = 'Android'

In [28]:
com_df.loc[(com_df['device_brand'] == 'Apple') & (com_df['device_category'] == 'mobile') & (com_df['device_os'].isna()), 'device_os'] = 'iOS'

In [29]:
com_df.loc[(com_df['device_brand'] == 'Apple') & (com_df['device_category'] == 'tablet') & (com_df['device_os'].isna()), 'device_os'] = 'iOS'

In [30]:
com_df['device_os'].value_counts(dropna=False)

device_os
Android          839442
other_os         397957
Windows          328815
iOS              186484
Macintosh         24156
Linux              4706
(not set)           309
Chrome OS            65
BlackBerry           24
Tizen                 7
Firefox OS            3
Nokia                 2
Samsung               2
Windows Phone         2
Name: count, dtype: int64

In [31]:
basic_os = [
    'Android',
    'iOS',
    'Windows',
    'Macintosh',
    'Linux',
    'other_os'
]

other_os_list = [i_os for i_os in com_df['device_os'].values if i_os not in basic_os]
other_os_list
com_df['device_os'] = com_df['device_os'].replace(other_os_list, 'other_os')
com_df['device_os'].value_counts(dropna=False)

device_os
Android      839442
other_os     398371
Windows      328815
iOS          186484
Macintosh     24156
Linux          4706
Name: count, dtype: int64

In [32]:
apple_desktop = com_df[(com_df['device_os'] == 'Macintosh')]
apple_desktop

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action
222,905648595358313255.1638525736.1638525736,2.108627e+08,2021-12-03,13:02:16,1,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,nSReTmyFtbSjlPrTKoaX,desktop,Macintosh,NaN,1440x900,Safari,Russia,Moscow,0
223,905648595358313255.1638525739.1638525739,2.108627e+08,2021-12-03,13:02:19,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,nSReTmyFtbSjlPrTKoaX,desktop,Macintosh,NaN,1440x900,Safari,Russia,Moscow,0
487,9057620093667786277.1635925543.1635925543,2.108892e+09,2021-11-03,10:45:43,1,kjsLglQLzykiRbcDiGcD,cpc,RoDitORHdzGfGhNCyEMy,NaN,QRSrGMRlRfLHqzjtnKsP,desktop,Macintosh,NaN,1440x900,Chrome,Russia,(not set),0
488,9057620093667786277.1635925543.1635925543,2.108892e+09,2021-11-03,10:45:43,1,kjsLglQLzykiRbcDiGcD,cpc,RoDitORHdzGfGhNCyEMy,NaN,QRSrGMRlRfLHqzjtnKsP,desktop,Macintosh,NaN,1440x900,Chrome,Russia,(not set),1
489,9057620093667786277.1635960751.1635960751,2.108892e+09,2021-11-03,20:32:31,2,kjsLglQLzykiRbcDiGcD,cpc,RoDitORHdzGfGhNCyEMy,NaN,QRSrGMRlRfLHqzjtnKsP,desktop,Macintosh,NaN,1440x900,Chrome,Russia,(not set),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781583,9053933615865089375.1637567841.1637567841,2.108033e+09,2021-11-22,10:57:21,1,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,desktop,Macintosh,NaN,1920x1080,Chrome,Russia,Saint Petersburg,0
1781625,9054082268976121089.1635517699.1635517699,2.108068e+09,2021-10-29,17:28:19,1,jaSOmLICuBzCFqHfBdRg,email,bxOTvPtyGSdUrbwoXCPO,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Macintosh,NaN,1280x800,Chrome,Russia,Voronezh,0
1781674,9054215168154192730.1640549210.1640549217,2.108099e+09,2021-12-26,23:06:57,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,UKNBOHebRIIsQRsjNKay,desktop,Macintosh,NaN,1792x1120,Safari,Russia,Kursk,0
1781744,9054511349090867625.1632491945.1632491945,2.108168e+09,2021-09-24,16:59:05,1,jaSOmLICuBzCFqHfBdRg,email,YCKgTzTDywjcWyQudGch,nNqUcgFgcqQbTVSvgaHr,puhZPIYqKXeFPaUviSjo,desktop,Macintosh,NaN,1792x1120,Safari,Russia,Moscow,0


In [33]:
com_df.loc[(com_df['device_os'] == 'Macintosh'), 'device_brand'] = 'Apple'

In [34]:
print(com_df['device_brand'].value_counts(dropna=False))

device_brand
Apple        542015
NaN          334247
Samsung      321626
Xiaomi       275796
Huawei       178330
              ...  
Star              1
Mobiistar         1
Smartfren         1
Tanix             1
Maxvi             1
Name: count, Length: 201, dtype: int64


In [35]:
com_df.loc[(com_df['device_brand'].isna()), 'device_brand'] = 'other_brand'

In [36]:
basic_brands = [
    'Apple',
    'Samsung',
    'Xiaomi',
    'Huawei',
    'other_brand',
]

other_brands = [brand for brand in com_df['device_brand'].values if brand not in basic_brands]

com_df['device_brand'] = com_df['device_brand'].replace(list(set(other_brands)), 'other_brand')
com_df['device_brand'].value_counts(dropna=False)

device_brand
Apple          542015
other_brand    464207
Samsung        321626
Xiaomi         275796
Huawei         178330
Name: count, dtype: int64

In [37]:
com_df['device_screen_resolution'].value_counts(dropna=False)

device_screen_resolution
414x896      159495
1920x1080    122688
393x851      110886
375x812      110099
360x780       89051
              ...  
1440x1075         1
1138x709          1
1762x905          1
1120x2328         1
464x1123          1
Name: count, Length: 4947, dtype: int64

In [38]:
com_df['screen_square'] = com_df['device_screen_resolution'].apply(lambda x: int(x.split('x')[0]) * int(x.split('x')[1]))
com_df['screen_square']

0           259200
1           328790
2           259200
3           308898
4           308898
            ...   
1781969    2073600
1781970     329160
1781971     250125
1781972     334443
1781973    1049088
Name: screen_square, Length: 1781974, dtype: int64

Изученине utm_

In [39]:
com_df['utm_source'].value_counts(dropna=False)

utm_source
ZpYIoDJMcFzVoPFsHGJL    568527
fDLlAcSmythWSCVMvqvL    287421
kjsLglQLzykiRbcDiGcD    251270
MvfHsxITijuriZxsqZqt    178039
BHcvLfOaCWvWTykYqHVe    114806
                         ...  
JdKLSdewxFqMMgKKiXBO         1
GYkgJdhSxirUFPHFiYSe         1
WRHPhoRKhKxaenRCLBfo         1
tjKUGseGMhkLEzdCqBRx         1
tYfqTBjAxWbmBruissGm         1
Name: count, Length: 281, dtype: int64

In [40]:
other_source = [source[0] for source in com_df['utm_source'].value_counts(dropna=False).items() if source[1] < 10**4]

com_df['utm_source'] = com_df['utm_source'].replace(list(set(other_source)), 'other_source')

com_df['utm_source'].value_counts(dropna=False)

utm_source
ZpYIoDJMcFzVoPFsHGJL    568527
fDLlAcSmythWSCVMvqvL    287421
kjsLglQLzykiRbcDiGcD    251270
MvfHsxITijuriZxsqZqt    178039
BHcvLfOaCWvWTykYqHVe    114806
bByPQxmDaMXgpHeypKSM     95864
other_source             85100
QxAxdyPLuQMEcrdZWdWb     46671
aXQzDWsJuGXeBXexNHjc     31337
jaSOmLICuBzCFqHfBdRg     28681
RmEBuqrriAfAVsLQQmhk     27550
PlbkrSYoHuZBWfYjYnfw     20410
vFcAhRxLfOWKhvxjELkx     18392
hTjLvqNxGggkGnxSCaTm     14573
gDBGzjFKYabGgSPZvrDH     13333
Name: count, dtype: int64

In [41]:
com_df['utm_medium'].value_counts(dropna=False)

utm_medium
banner               540289
cpc                  408734
(none)               287421
cpm                  232270
referral             144943
organic               57047
email                 28680
push                  28548
stories               10598
cpv                    7928
blogger_channel        7892
smartbanner            6680
blogger_stories        4261
tg                     3971
cpa                    3397
post                   2281
outlook                1288
smm                    1281
app                    1222
clicks                  949
blogger_header          762
(not set)               432
info_text               337
sms                     210
landing                 146
partner                  96
link                     53
cbaafe                   43
CPM                      39
yandex_cpc               32
vk_smm                   25
static                   18
google_cpc               15
article                  15
web_polka                11
fb_smm   

In [42]:

other_medium = [medium[0] for medium in com_df['utm_medium'].value_counts(dropna=False).items() if medium[1] < 10**4 or medium[0] == '(none)']

com_df['utm_medium'] = com_df['utm_medium'].replace(list(set(other_medium)), 'other_medium')

com_df['utm_medium'].value_counts(dropna=False)
    

utm_medium
banner          540289
cpc             408734
other_medium    330865
cpm             232270
referral        144943
organic          57047
email            28680
push             28548
stories          10598
Name: count, dtype: int64

In [43]:
com_df.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target_action,screen_square
0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,360x720,Chrome,Russia,Zlatoust,0,259200
1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,385x854,Samsung Internet,Russia,Moscow,0,328790
2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,360x720,Chrome,Russia,Krasnoyarsk,0,259200
3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,Android,Xiaomi,393x786,Chrome,Russia,Moscow,0,308898
4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,Android,Xiaomi,393x786,Chrome,Russia,Moscow,0,308898


In [44]:
com_df['geo_city'].value_counts(dropna=False).head(20)

geo_city
Moscow              774214
Saint Petersburg    285439
(not set)            74861
Yekaterinburg        34438
Krasnodar            31329
Kazan                28815
Samara               24156
Nizhny Novgorod      21326
Ufa                  20920
Novosibirsk          20622
Krasnoyarsk          15658
Chelyabinsk          15320
Tula                 15208
Rostov-on-Don        13458
Voronezh             12978
Irkutsk              12922
Grozny               12350
Balashikha           12226
Vladivostok          11760
Yaroslavl             9380
Name: count, dtype: int64

In [45]:
other_city = [city[0] for city in com_df['geo_city'].value_counts(dropna=False).items() if city[1] < 10**4 or city[0] == '(not set)']

com_df['geo_city'] = com_df['geo_city'].replace(list(set(other_city)), 'other_city')

com_df['geo_city'].value_counts(dropna=False).head(20)

geo_city
Moscow              774214
other_city          418835
Saint Petersburg    285439
Yekaterinburg        34438
Krasnodar            31329
Kazan                28815
Samara               24156
Nizhny Novgorod      21326
Ufa                  20920
Novosibirsk          20622
Krasnoyarsk          15658
Chelyabinsk          15320
Tula                 15208
Rostov-on-Don        13458
Voronezh             12978
Irkutsk              12922
Grozny               12350
Balashikha           12226
Vladivostok          11760
Name: count, dtype: int64

In [46]:
com_df['geo_country'].value_counts(dropna=False).head(20)

geo_country
Russia            1730983
Ukraine              8653
United States        8205
Belarus              3491
Germany              2162
Kazakhstan           2162
Ireland              1990
Turkey               1835
Sweden               1757
Netherlands          1489
Uzbekistan           1459
United Kingdom       1416
(not set)            1100
Kyrgyzstan            964
Georgia               909
France                771
Cyprus                732
Armenia               653
Finland               625
Spain                 568
Name: count, dtype: int64

In [48]:
other_country = [country[0] for country in com_df['geo_country'].value_counts(dropna=False).items() if country[1] < 10**4 or country[0] == '(not set)']

com_df['geo_country'] = com_df['geo_country'].replace(other_country, 'other_country')

com_df['geo_country'].value_counts(dropna=False).head()

geo_country
Russia           1730983
other_country      50991
Name: count, dtype: int64

In [49]:
com_df['year'] = com_df['visit_date'].apply(lambda x: int(x.split('-')[0]))

com_df['month'] = com_df['visit_date'].apply(lambda x: int(x.split('-')[1]))

com_df['day'] = com_df['visit_date'].apply(lambda x: int(x.split('-')[2]))

com_df['year'].head(5)


0    2021
1    2021
2    2021
3    2021
4    2021
Name: year, dtype: int64

In [50]:
com_df['hour'] = com_df['visit_time'].apply(lambda x: int(x.split(':')[0]))

com_df['hour']

0          14
1           8
2           2
3           5
4           5
           ..
1781969    10
1781970     8
1781971    15
1781972    15
1781973    19
Name: hour, Length: 1781974, dtype: int64

In [51]:
com_df.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,device_screen_resolution,device_browser,geo_country,geo_city,target_action,screen_square,year,month,day,hour
0,9055434745589932991.1637753792.1637753792,2.108383e+09,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,360x720,Chrome,Russia,other_city,0,259200,2021,11,24,14
1,905544597018549464.1636867290.1636867290,2.108385e+08,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,...,385x854,Samsung Internet,Russia,Moscow,0,328790,2021,11,14,8
2,9055446045651783499.1640648526.1640648526,2.108385e+09,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,360x720,Chrome,Russia,Krasnoyarsk,0,259200,2021,12,28,2
3,9055447046360770272.1622255328.1622255328,2.108386e+09,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,...,393x786,Chrome,Russia,Moscow,0,308898,2021,5,29,5
4,9055447046360770272.1622255345.1622255345,2.108386e+09,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,...,393x786,Chrome,Russia,Moscow,0,308898,2021,5,29,5


In [52]:
com_df.drop(columns=['client_id', 'visit_date', 'visit_time', 'utm_campaign', 'utm_adcontent', 'utm_keyword','device_screen_resolution'], axis=1, inplace=True)
com_df.head(5)

,session_id,visit_number,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city,target_action,screen_square,year,month,day,hour
0,9055434745589932991.1637753792.1637753792,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,other_city,0,259200,2021,11,24,14
1,905544597018549464.1636867290.1636867290,1,MvfHsxITijuriZxsqZqt,cpm,mobile,Android,Samsung,Samsung Internet,Russia,Moscow,0,328790,2021,11,14,8
2,9055446045651783499.1640648526.1640648526,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk,0,259200,2021,12,28,2
3,9055447046360770272.1622255328.1622255328,1,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,0,308898,2021,5,29,5
4,9055447046360770272.1622255345.1622255345,2,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,0,308898,2021,5,29,5


In [53]:
com_df.to_csv('data/clean_df.csv', index=False)

## Feature engineering

### Преобразование категориальных переменных

In [54]:
com_df.describe(include=[object])

,session_id,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city
count,1781974,1781974,1781974,1781974,1781974,1781974,1781974,1781974,1781974
unique,1732266,15,9,3,6,5,55,2,19
top,1906362532065501532.1625350494.1625350494,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Apple,Chrome,Russia,Moscow
freq,2,568527,540289,1406765,839442,542015,978405,1730983,774214


In [55]:
cat_variables = [category_var for category_var in com_df.describe(include=[object]).keys() if category_var != 'session_id']
cat_variables

['utm_source',
 'utm_medium',
 'device_category',
 'device_os',
 'device_brand',
 'device_browser',
 'geo_country',
 'geo_city']

In [56]:
for variable in cat_variables:
    print(
        f"Количество уникальных значений {com_df[variable].name}: {len(com_df[variable].unique())}"
    )

Количество уникальных значений utm_source: 15
Количество уникальных значений utm_medium: 9
Количество уникальных значений device_category: 3
Количество уникальных значений device_os: 6
Количество уникальных значений device_brand: 5
Количество уникальных значений device_browser: 55
Количество уникальных значений geo_country: 2
Количество уникальных значений geo_city: 19


Создаю дадафрейм только с необходимыми переменными

In [57]:
data = com_df[cat_variables]
data

,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city
0,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,other_city
1,MvfHsxITijuriZxsqZqt,cpm,mobile,Android,Samsung,Samsung Internet,Russia,Moscow
2,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk
3,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow
4,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...
1781969,BHcvLfOaCWvWTykYqHVe,cpc,desktop,Windows,other_brand,Chrome,Russia,Moscow
1781970,fDLlAcSmythWSCVMvqvL,other_medium,mobile,other_os,Apple,Safari,Russia,other_city
1781971,fDLlAcSmythWSCVMvqvL,other_medium,mobile,iOS,Apple,Safari,Russia,Moscow
1781972,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Xiaomi,Chrome,Russia,Chelyabinsk


Преобразую катгориальные переменные с помощью OnrHotEncoder

In [58]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(data[cat_variables])
ohe_data = ohe.transform(data)
print(ohe_data.shape)
ohe_data

(1781974, 114)


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Дополняю датафрейм перобразованными переменными

In [59]:
com_df[ohe.get_feature_names_out()] = ohe_data
com_df.head(5)

/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_27222/1171158276.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  com_df[ohe.get_feature_names_out()] = ohe_data
/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_27222/1171158276.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  com_df[ohe.get_feature_names_out()] = ohe_data
/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_27222/1171158276.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

,session_id,visit_number,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city,...,geo_city_Novosibirsk,geo_city_Rostov-on-Don,geo_city_Saint Petersburg,geo_city_Samara,geo_city_Tula,geo_city_Ufa,geo_city_Vladivostok,geo_city_Voronezh,geo_city_Yekaterinburg,geo_city_other_city
0,9055434745589932991.1637753792.1637753792,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,other_city,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,905544597018549464.1636867290.1636867290,1,MvfHsxITijuriZxsqZqt,cpm,mobile,Android,Samsung,Samsung Internet,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9055446045651783499.1640648526.1640648526,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9055447046360770272.1622255328.1622255328,1,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9055447046360770272.1622255345.1622255345,2,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Преобразую количественные переменные

In [62]:
count_variables = [
    'visit_number',
    'year',
    'month',
    'day',
    'hour',
    'screen_square',
]

In [63]:
columns_std = [f'{col}_std' for col in count_variables]

data_num = com_df[count_variables]
data_num

,visit_number,year,month,day,hour,screen_square
0,1,2021,11,24,14,259200
1,1,2021,11,14,8,328790
2,1,2021,12,28,2,259200
3,1,2021,5,29,5,308898
4,2,2021,5,29,5,308898
...,...,...,...,...,...,...
1781969,1,2021,12,22,10,2073600
1781970,1,2021,5,26,8,329160
1781971,1,2021,11,15,15,250125
1781972,1,2021,11,29,15,334443


Для преобразования количественных переменных использую StandartScaler

In [64]:
std_scaler = StandardScaler()
std_scaler.fit(data_num)
std_scaler_new = std_scaler.transform(data_num)
std_scaler_new


array([[-0.13693256,  0.        ,  0.78446228,  0.78509135,  0.06310443,
        -0.49773226],
       [-0.13693256,  0.        ,  0.78446228, -0.35209057, -0.91636661,
        -0.3988602 ],
       [-0.13693256,  0.        ,  1.23577526,  1.23996413, -1.89583765,
        -0.49773226],
       ...,
       [-0.13693256,  0.        ,  0.78446228, -0.23837238,  0.22634961,
        -0.51062584],
       [-0.13693256,  0.        ,  0.78446228,  1.35368232,  0.22634961,
        -0.39082853],
       [-0.13693256,  0.        ,  1.23577526,  1.5811187 ,  0.8793303 ,
         0.62452461]])

Добавляю новые фичи в исходный датасет

In [65]:
com_df[columns_std] = std_scaler_new
print(com_df.columns.to_list())
com_df.head(5)

['session_id', 'visit_number', 'utm_source', 'utm_medium', 'device_category', 'device_os', 'device_brand', 'device_browser', 'geo_country', 'geo_city', 'target_action', 'screen_square', 'year', 'month', 'day', 'hour', 'utm_source_BHcvLfOaCWvWTykYqHVe', 'utm_source_MvfHsxITijuriZxsqZqt', 'utm_source_PlbkrSYoHuZBWfYjYnfw', 'utm_source_QxAxdyPLuQMEcrdZWdWb', 'utm_source_RmEBuqrriAfAVsLQQmhk', 'utm_source_ZpYIoDJMcFzVoPFsHGJL', 'utm_source_aXQzDWsJuGXeBXexNHjc', 'utm_source_bByPQxmDaMXgpHeypKSM', 'utm_source_fDLlAcSmythWSCVMvqvL', 'utm_source_gDBGzjFKYabGgSPZvrDH', 'utm_source_hTjLvqNxGggkGnxSCaTm', 'utm_source_jaSOmLICuBzCFqHfBdRg', 'utm_source_kjsLglQLzykiRbcDiGcD', 'utm_source_other_source', 'utm_source_vFcAhRxLfOWKhvxjELkx', 'utm_medium_banner', 'utm_medium_cpc', 'utm_medium_cpm', 'utm_medium_email', 'utm_medium_organic', 'utm_medium_other_medium', 'utm_medium_push', 'utm_medium_referral', 'utm_medium_stories', 'device_category_desktop', 'device_category_mobile', 'device_category_table

/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_27222/1822451452.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  com_df[columns_std] = std_scaler_new
/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_27222/1822451452.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  com_df[columns_std] = std_scaler_new
/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_27222/1822451452.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

,session_id,visit_number,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city,...,geo_city_Vladivostok,geo_city_Voronezh,geo_city_Yekaterinburg,geo_city_other_city,visit_number_std,year_std,month_std,day_std,hour_std,screen_square_std
0,9055434745589932991.1637753792.1637753792,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,other_city,...,0.0,0.0,0.0,1.0,-0.136933,0.0,0.784462,0.785091,0.063104,-0.497732
1,905544597018549464.1636867290.1636867290,1,MvfHsxITijuriZxsqZqt,cpm,mobile,Android,Samsung,Samsung Internet,Russia,Moscow,...,0.0,0.0,0.0,0.0,-0.136933,0.0,0.784462,-0.352091,-0.916367,-0.398860
2,9055446045651783499.1640648526.1640648526,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk,...,0.0,0.0,0.0,0.0,-0.136933,0.0,1.235775,1.239964,-1.895838,-0.497732
3,9055447046360770272.1622255328.1622255328,1,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,...,0.0,0.0,0.0,0.0,-0.136933,0.0,-1.923416,1.353682,-1.406102,-0.427122
4,9055447046360770272.1622255345.1622255345,2,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,...,0.0,0.0,0.0,0.0,-0.055232,0.0,-1.923416,1.353682,-1.406102,-0.427122


Удаляю лишние колонки (переменные без преобразования)

In [69]:
columns_for_drop = cat_variables.copy()
columns_for_drop


df_prepared = com_df.drop(columns=columns_for_drop)
df_prepared

,session_id,target_action,utm_source_BHcvLfOaCWvWTykYqHVe,utm_source_MvfHsxITijuriZxsqZqt,utm_source_PlbkrSYoHuZBWfYjYnfw,utm_source_QxAxdyPLuQMEcrdZWdWb,utm_source_RmEBuqrriAfAVsLQQmhk,utm_source_ZpYIoDJMcFzVoPFsHGJL,utm_source_aXQzDWsJuGXeBXexNHjc,utm_source_bByPQxmDaMXgpHeypKSM,...,geo_city_Vladivostok,geo_city_Voronezh,geo_city_Yekaterinburg,geo_city_other_city,visit_number_std,year_std,month_std,day_std,hour_std,screen_square_std
0,9055434745589932991.1637753792.1637753792,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.136933,0.0,0.784462,0.785091,0.063104,-0.497732
1,905544597018549464.1636867290.1636867290,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,0.784462,-0.352091,-0.916367,-0.398860
2,9055446045651783499.1640648526.1640648526,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,1.235775,1.239964,-1.895838,-0.497732
3,9055447046360770272.1622255328.1622255328,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,-1.923416,1.353682,-1.406102,-0.427122
4,9055447046360770272.1622255345.1622255345,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.055232,0.0,-1.923416,1.353682,-1.406102,-0.427122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781969,9055415581448263752.1640159305.1640159305,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,1.235775,0.557655,-0.589876,2.080131
1781970,9055421130527858185.1622007305.1622007305,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.136933,0.0,-1.923416,1.012528,-0.916367,-0.398335
1781971,9055422955903931195.1636979515.1636979515,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,0.784462,-0.238372,0.226350,-0.510626
1781972,905543020766873816.1638189404.1638189404,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,0.784462,1.353682,0.226350,-0.390829


In [70]:
print(df_prepared.shape)
df_prepared.columns.to_list()

(1781974, 122)


['session_id',
 'target_action',
 'utm_source_BHcvLfOaCWvWTykYqHVe',
 'utm_source_MvfHsxITijuriZxsqZqt',
 'utm_source_PlbkrSYoHuZBWfYjYnfw',
 'utm_source_QxAxdyPLuQMEcrdZWdWb',
 'utm_source_RmEBuqrriAfAVsLQQmhk',
 'utm_source_ZpYIoDJMcFzVoPFsHGJL',
 'utm_source_aXQzDWsJuGXeBXexNHjc',
 'utm_source_bByPQxmDaMXgpHeypKSM',
 'utm_source_fDLlAcSmythWSCVMvqvL',
 'utm_source_gDBGzjFKYabGgSPZvrDH',
 'utm_source_hTjLvqNxGggkGnxSCaTm',
 'utm_source_jaSOmLICuBzCFqHfBdRg',
 'utm_source_kjsLglQLzykiRbcDiGcD',
 'utm_source_other_source',
 'utm_source_vFcAhRxLfOWKhvxjELkx',
 'utm_medium_banner',
 'utm_medium_cpc',
 'utm_medium_cpm',
 'utm_medium_email',
 'utm_medium_organic',
 'utm_medium_other_medium',
 'utm_medium_push',
 'utm_medium_referral',
 'utm_medium_stories',
 'device_category_desktop',
 'device_category_mobile',
 'device_category_tablet',
 'device_os_Android',
 'device_os_Linux',
 'device_os_Macintosh',
 'device_os_Windows',
 'device_os_iOS',
 'device_os_other_os',
 'device_brand_Apple',
 '

In [80]:
df_prepared['target_action'].value_counts()

target_action
0    1731660
1      50314
Name: count, dtype: int64

Сохраняю финальный датасет в новый файл

In [71]:
df_prepared.to_csv('data/final_prepared.csv', index=False)

# Modelling

### 1. RandomForest

Формирую датасет для обучения и делю его на train и test

In [81]:
df_prep = pd.read_csv('data/final_prepared.csv')

In [92]:
target_0 = df_prep[df_prep.target_action == 0]
target_1 = df_prep[df_prep.target_action == 1]

target_0_downsampled = resample(
    target_0,
    replace = False,
    n_samples = len(target_1),
    random_state = 27,
    )


downsampled = pd.concat([target_0_downsampled, target_1])
downsampled

,session_id,target_action,utm_source_BHcvLfOaCWvWTykYqHVe,utm_source_MvfHsxITijuriZxsqZqt,utm_source_PlbkrSYoHuZBWfYjYnfw,utm_source_QxAxdyPLuQMEcrdZWdWb,utm_source_RmEBuqrriAfAVsLQQmhk,utm_source_ZpYIoDJMcFzVoPFsHGJL,utm_source_aXQzDWsJuGXeBXexNHjc,utm_source_bByPQxmDaMXgpHeypKSM,...,geo_city_Vladivostok,geo_city_Voronezh,geo_city_Yekaterinburg,geo_city_other_city,visit_number_std,year_std,month_std,day_std,hour_std,screen_square_std
764548,4322078771880776944.1630917875.1630917875,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,-0.118164,-1.261836,-0.426631,2.916686
1650983,8448224922257355185.1640001969.1640001981,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,-0.136933,0.0,1.235775,0.330219,0.226350,-0.433371
1772392,9010399866005200270.1635229071.1635229071,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,0.333149,1.012528,-0.753121,-0.538651
1104941,5904861657402899665.1632828625.1632828625,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.136933,0.0,-0.118164,1.239964,0.063104,-0.467043
1045106,5626349258930766250.1624883626.1624883626,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,-1.472103,1.239964,0.226350,2.080131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1781856,9054912872702758664.1636781836.1636781836,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,0.784462,-0.465809,-0.916367,-0.456814
1781930,9055248417020050011.1634073188.1634073188,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.136933,0.0,0.333149,-0.465809,-2.222328,-0.467043
1781933,9055248417020050011.1634075541.1634075541,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.026468,0.0,0.333149,-0.465809,-2.222328,-0.467043
1781961,9055376699099939975.1630766214.1630766214,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,-0.136933,0.0,-0.118164,-1.489273,0.552840,2.080131


In [111]:
df_forest = downsampled.copy()
df_forest = df_forest.drop(['session_id', 'year_std'], axis=1)

x = df_forest.drop(['target_action'], axis=1)
y = df_forest['target_action']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42
)

Обучаю модель RandomForest

In [112]:
rf_clf = RandomForestClassifier(
    n_estimators=150, 
    min_samples_split=3, 
    bootstrap=False, 
    random_state=42, 
    max_depth=80,
)

rf_clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=80, min_samples_split=3,
                       n_estimators=150, random_state=42)

In [113]:
predicted_train_rf = rf_clf.predict(x_train)
predicted_test_rf = rf_clf.predict(x_test)

print(accuracy_score(y_train, predicted_train_rf))
print(accuracy_score(y_test, predicted_test_rf))

0.9811893979187666
0.5931299479943025


Кросс-валидация

In [100]:
cross_validate(rf_clf, x, y, cv=5)

{'fit_time': array([19.49386501, 19.30979204, 19.29348779, 19.78207707, 19.53355169]),
 'score_time': array([0.99786592, 0.9529562 , 0.95773005, 0.94069386, 0.92567229]),
 'test_score': array([0.58387161, 0.58675345, 0.58203319, 0.58385093, 0.58231056])}

In [16]:
cv_score = cross_val_score(rf_clf, x, y, cv=5)

In [17]:
print(cv_score.mean())
print(cv_score.std())

0.7685304330534013
0.0093588363279649


### 2. LogisticRegression

Формирую датасет для обучения и делю его на train и test

In [93]:
logreg_df = downsampled.copy()
x = logreg_df.drop(['target_action', 'session_id', 'year_std'], axis=1)
y = logreg_df['target_action']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

Обучаю модель логистической регрессии

In [109]:
lr = LogisticRegression(
    C=4,
    max_iter=154,
    random_state=46,
    solver='liblinear',
    )
lr.fit(x_train, y_train)

LogisticRegression(C=4, max_iter=154, random_state=46, solver='liblinear')

In [110]:
logreg_pred_train = lr.predict(x_train)
print('accuracy train', accuracy_score(y_train, logreg_pred_train))

logreg_pred_test = lr.predict(x_test)
print('accuracy test' ,accuracy_score(y_test, logreg_pred_test))
print('confusion_matrix', confusion_matrix(y_test, logreg_pred_test))

accuracy train 0.6089950169650336
accuracy test 0.6141972241544934
confusion_matrix [[8922 6239]
 [5408 9620]]


In [102]:
from sklearn.model_selection import GridSearchCV
param_grid = {
   'max_iter': list(range(150, 501, 50)),
   'solver': ['liblinear', 'lbfgs', 'saga', 'newton-cg'],
   'C': list(range(1, 10)),
   'random_state': list(range(10, 101, 10)),
   # 'verbose': [1, 0]
}
new_lr = LogisticRegression(random_state=42)
grid_search_lr = GridSearchCV(
   estimator=new_lr,
   param_grid=param_grid,
   scoring='accuracy',
   verbose=1,
   n_jobs=-1
)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

grid_search_lr.fit(x_train, y_train)

best_params = grid_search_lr.best_params_
best_params

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits


/Users/igorkolotuskin/Desktop/Python projects/ds_diploma/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/igorkolotuskin/Desktop/Python projects/ds_diploma/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/igorkolotuskin/Desktop/Python projects/ds_diploma/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/igorkolotuskin/Desktop/Python projects/ds_diploma/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/igorkolotuskin/Desktop/Python projects/ds_diploma/venv/lib/python3.11/site-packages/sklea

{'C': 4, 'max_iter': 150, 'random_state': 10, 'solver': 'lbfgs'}

In [96]:
cv_score = cross_val_score(lr, x, y, cv=5)
print(cv_score.mean())
print(cv_score.std())

/Users/igorkolotuskin/Desktop/Python projects/ds_diploma/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/igorkolotuskin/Desktop/Python projects/ds_diploma/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opti

0.6086079883072051
0.003756113042659759


/Users/igorkolotuskin/Desktop/Python projects/ds_diploma/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 3. Многослойный персептрон

Формирую датасет для обучения и делю его на train и test

In [26]:
mlp_df = df_prep.copy()
x = mlp_df.drop(['price_category', 'price', 'id'], axis=1)
y = mlp_df['price_category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [27]:
mlp = MLPClassifier(
    random_state=42, 
    max_iter=500, 
    hidden_layer_sizes=(100,40),
    )
mlp.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(100, 40), max_iter=500, random_state=42)

In [28]:
mlp_pred_train = mlp.predict(x_train)
print('accuracy train' ,accuracy_score(y_train, mlp_pred_train))

mlp_pred_test = mlp.predict(x_test)
print('accuracy test' ,accuracy_score(y_test, mlp_pred_test))

accuracy train 0.9971144135045448
accuracy test 0.7458768091551666


In [30]:
cv_score = cross_val_score(mlp, x, y, cv=5)
print(cv_score.mean())
print(cv_score.std())

0.7411609787934876
0.012725531046048847


# Results

В моем случае лучше всего показала себя модель random forest.
По результатам кросс-вализации:
1. accuracy = 0.76
2. std = 0.009

Обучение модели на всем датасете

In [30]:
df_forest = df_prep.copy()
df_forest = df_forest.drop(['id', 'price'], axis=1)

x = df_forest.drop(['price_category'], axis=1)
y = df_forest['price_category']

In [31]:
rf_clf = RandomForestClassifier(
    n_estimators=150, 
    min_samples_split=3, 
    bootstrap=False, 
    random_state=42, 
    max_depth=80,
)

rf_clf.fit(x, y)

RandomForestClassifier(bootstrap=False, max_depth=80, min_samples_split=3,
                       n_estimators=150, random_state=42)

Сохраняю полученню модель

In [32]:
with open('model.pickle', 'wb') as file:
    pickle.dump(rf_clf, file)